In [1]:
'''
narrowing SIU reports from 2017/18 to only those with the following keywords
the list of all the reports to scrape are in an csv file in reports_to_scrape folder downloaded from their site
Ontario - Special Investigations Unit (SIU Annual Report 2021-2022 and Special Investigations Unit (SIU) director’s reports from 2005 to November 2018 | Ontario.ca) 
Firearm injuries
Custody injuries
Other injuries/deaths
Vehicle injuries
Use of force
Unlawful or unnecessary exercise of authority 
Unnecessary force
Unnecessary use of force
Assault
Sexual assault
Harm
Excessive force
Injury
'''

# import libraries
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [2]:
# import pages to scrape from the csv file
df = pd.read_csv("reports_to_scrape/on_siu_2005_2017.csv")
df.head()

,Case number,Special Investigations Unit incident date,Special Investigations Unit published date
0,"<a href=""/page/siu-directors-report-case-15-tf...",2015-07-25,2016-06-29
1,"<a href=""/page/siu-directors-report-case-09-of...",2009-04-22,2009-07-07
2,"<a href=""/page/siu-directors-report-case-05-tf...",2005-05-27,2005-08-25
3,"<a href=""/page/siu-directors-report-case-05-of...",2005-10-18,2005-12-07
4,"<a href=""/page/siu-directors-report-case-06-of...",2006-04-13,2006-07-24


In [3]:
len(df)

79

In [4]:
# base url for page is: https://www.ontario.ca/
baseurl = "https://www.ontario.ca"

In [5]:
# get the link from the case number part of the df

cases = df["Case number"].tolist()
index = 0
for case in cases:
    # print(case)

    soup = BeautifulSoup(case, "html.parser")
    link = soup.find("a")['href']

    # print(link)
    url = baseurl + link
    
    # scrape the first directors report 17-OFD-379
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    # print(soup.find("title"))
    
    # find if the page contains any of the keywords
    contains_words = False
    keywords = ["Firearm injuries", "Custody injuries", "Other injuries/deaths", "Vehicle injuries", "Use of force", 
                "Unlawful or unnecessary exercise of authority", "Unnecessary force", "Unnecessary use of force",
                "Assault", "Sexual assault", "Harm", "Excessive force", "Injury"]

    for i in keywords:
        results = soup.body.find_all(string=re.compile('.*{0}.*'.format(i)), recursive=True)
        # print('Found the word "{0}" {1} times\n'.format(i, len(results)))
        if len(results) > 0:
            contains_words = True
    
    if contains_words == False:
        df = df.drop(index)
    
    index += 1

In [6]:
df.head

<bound method NDFrame.head of                                           Case number  \
2   <a href="/page/siu-directors-report-case-05-tf...   
12  <a href="/page/siu-directors-report-case-15-of...   
14  <a href="/page/siu-directors-report-case-10-pf...   
25  <a href="/page/siu-directors-report-case-10-of...   
32  <a href="/page/siu-directors-report-case-11-of...   
47  <a href="/page/siu-directors-report-case-09-tf...   
59  <a href="/page/siu-directors-report-case-13-pf...   
70  <a href="/page/siu-directors-report-case-15-tc...   
72  <a href="/page/siu-directors-report-case-10-oo...   
74  <a href="/page/siu-directors-report-case-13-tc...   

   Special Investigations Unit incident date  \
2                                 2005-05-27   
12                                2015-04-04   
14                                2010-05-10   
25                                2010-08-25   
32                                2011-06-22   
47                                2009-12-27   
59    

In [7]:
len(df)

10

In [8]:
df.to_csv("reports_to_scrape/on_siu_2005_2017_w_keywords.csv")